In [ ]:
# import the necessary packages
import cv2
import numpy as np
from imutils.perspective import four_point_transform
from imutils import contours
width = 1000
height = 1000
question = 10
answer =  4
ans = [2, 1, 0, 3, 0, 3, 3, 1, 2, 1]
position_to_char = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

def  getCornerPoint(cont):
    peri = cv2.arcLength(cont, True)
    approx =cv2.approxPolyDP(cont, 0.02*peri,True)
    return approx
def reorder(myPoint):
    myPoint = myPoint.reshape((4,2))
    myPointNew = np.zeros((4,1,2),np.int32)
    add = myPoint.sum(1)
    #print (myPoint)
    #print (add)
    myPointNew[0]= myPoint[np.argmin(add)]    
    myPointNew[3]= myPoint[np.argmax(add)]
    diff = np.diff(myPoint, axis=1)
    myPointNew[1]= myPoint[np.argmin(diff)]
    myPointNew[2]= myPoint[np.argmax(diff)]
    #print(diff)    
    return myPointNew

def splitBoxes(img):
     rows = np.vsplit(img, 10)
     boxes = []
     for r in rows:
        cols = np.hsplit(r,4)
        for box in cols:
            boxes.append(box)
           #cv2.imshow("Split",rows[0])
     return boxes

path = "giuaky3.jfif"
image = cv2.imread(path)
image = cv2.resize (image,(width,height))
imgBiggestContours = image.copy()
imgContours = image.copy()
cv2.imshow('Original', image)
cv2.waitKey(0)

# Use the cvtColor() function to grayscale the image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('Grayscale', gray_image)
cv2.waitKey(0)  

blur_image = cv2.GaussianBlur(gray_image,(5,5),1)
cv2.imshow('Blur image', blur_image)
cv2.waitKey(0)

Canny_image = cv2.Canny(blur_image,10,50)
cv2.imshow('Canny image', Canny_image)
cv2.waitKey(0)
# Window shown waits for any key pressing event
countours, hierarchy = cv2.findContours (Canny_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cv2.drawContours (imgContours, countours, -1,(0,255,0),10) 
imgBlank = np.zeros_like(image)
cv2.imshow('imgBlank',imgContours)
cv2.waitKey(0)
rectCon= []
for i in countours:
        area = cv2.contourArea(i)
        #print ("Area", area)
        if area>50:
            peri = cv2.arcLength(i,True)
            approx = cv2.approxPolyDP(i,0.02*peri,True) 
            #print("Corner Point: ", len(approx))
            if len(approx)==4:
                  rectCon.append(i)
rectCon= sorted(rectCon, key= cv2.contourArea, reverse=True)
biggestContour = getCornerPoint(rectCon[0])
#print (biggestContour.shape)
#print (biggestContour)
if biggestContour.size != 0:
      cv2.drawContours(imgBiggestContours, biggestContour, -1,(0,255,0),10)
cv2.imshow("biggestContour",imgBiggestContours)
cv2.waitKey(0)
biggestContour = reorder(biggestContour)
pt1 = np.float32(biggestContour)
pt2= np.float32([[0,0],[width,0],[0,height],[width,height]])

matrix = cv2.getPerspectiveTransform(pt1,pt2)
imgwarpColored = cv2.warpPerspective (image,matrix,(width,height))
cv2.imshow ("thang dung",imgwarpColored)
cv2.waitKey (0)
# Chia hai tam anh
(height, width, channels) = imgwarpColored.shape
imgwarpColored1= imgwarpColored[:, (width*6)//10:]
cv2.imshow("nua phai",imgwarpColored1)
imgwarpColored2= imgwarpColored[:,(width)//10 :(width)//2]
cv2.imshow("nua trai",imgwarpColored2)
cv2.waitKey(0)
# Apply Threshold
imgWarpGray1 = cv2.cvtColor(imgwarpColored1,cv2.COLOR_BGR2GRAY)
imgThresh1 = cv2.threshold(imgWarpGray1,140,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imshow ("threshold phai",imgThresh1)
cv2.waitKey(0)
boxes1 = splitBoxes(imgThresh1)
print(cv2.countNonZero(boxes1[0]),cv2.countNonZero(boxes1[2]))

imgWarpGray2 = cv2.cvtColor(imgwarpColored2,cv2.COLOR_BGR2GRAY)
imgThresh2 = cv2.threshold(imgWarpGray2,100,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imshow ("threshold trai",imgThresh2)
cv2.waitKey(0)
boxes2 = splitBoxes(imgThresh2)
print(cv2.countNonZero(boxes2[0]),cv2.countNonZero(boxes2[2]))



myPixelVal1 = np.zeros((question,answer))
countR1 = 0
countC1 = 0

for im in boxes1:
    totalPixel1 = cv2.countNonZero(im)
    if countR1 < question and countC1 < answer:
        myPixelVal1[countR1][countC1] = totalPixel1
        countC1 += 1
    if countC1 == answer:
        countC1 = 0
        countR1 += 1 
print(myPixelVal1)

myIndex1 = []
for x in range (0,question):
     arr1 = myPixelVal1[x]
     #print("arr",arr)``
     myIndexVal1 = np.where(arr1 == np.amax(arr1))
     #print (myIndexVal[0])
     myIndex1.append(myIndexVal1[0][0])
print(myIndex1)

myPixelVal2 = np.zeros((question,answer))
countR2 = 0
countC2 = 0

for im in boxes2:
    totalPixel2 = cv2.countNonZero(im)
    if countR2 < question and countC2 < answer:
        myPixelVal2[countR2][countC2] = totalPixel2
        countC2 += 1
    if countC2 == answer:
        countC2 = 0
        countR2 += 1 
print(myPixelVal2)

myIndex2 = []
for x in range (0,question):
     arr2 = myPixelVal2[x]
     #print("arr",arr)``
     myIndexVal2 = np.where(arr2 == np.amax(arr2))
     #print (myIndexVal[0])
     myIndex2.append(myIndexVal2[0][0])
print(myIndex2)
cv2.waitKey(0)

# Sử dụng danh sách để lưu kết quả sau khi chuyển đổi
print("Cau 1 den 10:/n")
char_list2 = [position_to_char[pos] for pos in myIndex2]
print (char_list2)
print ("Cau 10 den 20:/n ")
char_list1 = [position_to_char[pos] for pos in myIndex1]
print (char_list1)
cv2.waitKey(0)


549 719
1046 753
[[ 549.  592.  719.  908.]
 [ 776.  941.  802.  856.]
 [2012. 1114. 1932. 1213.]
 [ 969.  938. 1027. 1438.]
 [1316. 1033. 1111. 1198.]
 [ 997. 1217. 1177. 1807.]
 [ 991. 1245. 1571. 1255.]
 [1068. 1693. 1378. 1413.]
 [2183. 1389. 2532. 2522.]
 [1344. 1488. 1936. 1430.]]
[3, 1, 0, 3, 0, 3, 2, 1, 2, 2]
[[1046.  894.  753.  835.]
 [ 962. 1433.  907. 1681.]
 [ 998.  963. 1530. 1171.]
 [ 905.  956. 1054. 1589.]
 [1265. 1255. 1457. 1239.]
 [1280. 1119. 1312. 1123.]
 [1272. 1205. 1381. 1213.]
 [1345. 1615. 1214. 1333.]
 [1303. 1267. 1281. 1800.]
 [1681. 1365. 1416. 1626.]]
[0, 3, 2, 3, 2, 2, 2, 1, 3, 0]
Cau 1 den 10:/n
['A', 'D', 'C', 'D', 'C', 'C', 'C', 'B', 'D', 'A']
Cau 10 den 20:/n 
['D', 'B', 'A', 'D', 'A', 'D', 'C', 'B', 'C', 'C']
